# Importing requirements

#### The following notebook was written by BM22BTECH11004 for preprocessing the PTBXL dataset and running main models on the dataset

In [2]:
pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import pandas as pd
import wfdb
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from collections import Counter
import time
from tqdm import tqdm


# Loading Data from dataset file

In [4]:
DATA_PATH = '/home/bmi-lab/Downloads/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3'

ptbxl_df = pd.read_csv(os.path.join(DATA_PATH, 'ptbxl_database.csv'))
scp_statements = pd.read_csv(os.path.join(DATA_PATH, 'scp_statements.csv'), index_col=0)

diagnostic_scps = scp_statements[scp_statements['diagnostic'] == 1].index.values

scp_to_superclass = scp_statements['diagnostic_class'].to_dict()
scp_to_subclass = scp_statements['diagnostic_subclass'].to_dict()

In [5]:
ptbxl_df['scp_codes'] = ptbxl_df['scp_codes'].apply(lambda x: ast.literal_eval(x))

In [6]:
def aggregate_diagnostic_labels(df, scp_codes, scp_to_agg):
    df = df.copy()
    def aggregate_labels(scp_codes_dict):
        labels = set()
        for code in scp_codes_dict.keys():
            if code in scp_codes:
                label = scp_to_agg.get(code)
                if label:
                    labels.add(label)
        return list(labels)
    df['diagnostic_labels'] = df['scp_codes'].apply(aggregate_labels)
    return df

ptbxl_df = aggregate_diagnostic_labels(ptbxl_df, diagnostic_scps, scp_to_superclass)
ptbxl_df = ptbxl_df.rename(columns={'diagnostic_labels': 'superclass_labels'})

ptbxl_df = aggregate_diagnostic_labels(ptbxl_df, diagnostic_scps, scp_to_subclass)
ptbxl_df = ptbxl_df.rename(columns={'diagnostic_labels': 'subclass_labels'})

In [7]:
ptbxl_df = ptbxl_df[ptbxl_df['superclass_labels'].map(len) > 0]

In [8]:
train_df = ptbxl_df[ptbxl_df.strat_fold <= 8]
val_df = ptbxl_df[ptbxl_df.strat_fold == 9]
test_df = ptbxl_df[ptbxl_df.strat_fold == 10]

In [9]:
def load_data(df, sampling_rate, data_path):
    data = []
    if sampling_rate == 100:
        filenames = df['filename_lr'].values
    else:
        filenames = df['filename_hr'].values
    for filename in filenames:
        file_path = os.path.join(data_path, filename)
        signals, _ = wfdb.rdsamp(file_path)
        data.append(signals)
    return np.array(data)

X_train = load_data(train_df, sampling_rate=100, data_path=DATA_PATH)
X_val = load_data(val_df, sampling_rate=100, data_path=DATA_PATH)
X_test = load_data(test_df, sampling_rate=100, data_path=DATA_PATH)

In [10]:
train_labels_super = train_df['superclass_labels'].values
val_labels_super = val_df['superclass_labels'].values
test_labels_super = test_df['superclass_labels'].values

mlb_super = MultiLabelBinarizer()
y_train_super = mlb_super.fit_transform(train_labels_super)
y_val_super = mlb_super.transform(val_labels_super)
y_test_super = mlb_super.transform(test_labels_super)
classes_super = mlb_super.classes_

In [11]:
train_labels_sub = train_df['subclass_labels'].values
val_labels_sub = val_df['subclass_labels'].values
test_labels_sub = test_df['subclass_labels'].values

mlb_sub = MultiLabelBinarizer()
y_train_sub = mlb_sub.fit_transform(train_labels_sub)
y_val_sub = mlb_sub.transform(val_labels_sub)
y_test_sub = mlb_sub.transform(test_labels_sub)
classes_sub = mlb_sub.classes_

In [12]:
def normalize_data_per_channel(X):
    X = np.transpose(X, (0, 2, 1))
    mean = np.mean(X, axis=(0, 2), keepdims=True)
    std = np.std(X, axis=(0, 2), keepdims=True)
    X = (X - mean) / std
    X = np.transpose(X, (0, 2, 1))
    return X

X_train = normalize_data_per_channel(X_train)
X_val = normalize_data_per_channel(X_val)
X_test = normalize_data_per_channel(X_test)

In [13]:
class_counts_super = np.sum(y_train_super, axis=0)
total_samples_super = y_train_super.shape[0]

class_weight_super = {}
for i, count in enumerate(class_counts_super):
    class_weight_super[i] = total_samples_super / (len(class_counts_super) * count)

class_counts_sub = np.sum(y_train_sub, axis=0)
total_samples_sub = y_train_sub.shape[0]

class_weight_sub = {}
for i, count in enumerate(class_counts_sub):
    class_weight_sub[i] = total_samples_sub / (len(class_counts_sub) * count)

In [14]:
num_classes_super = y_train_super.shape[1]
class_totals = np.sum(y_train_super, axis=0)
class_weights = class_totals.max() / class_totals
weights_array = np.array(class_weights, dtype=np.float32)

In [15]:
num_classes_sub = y_train_sub.shape[1]
class_totals_sub = np.sum(y_train_sub, axis=0)
class_weights_sub = class_totals_sub.max() / class_totals_sub
weights_array_sub = np.array(class_weights_sub, dtype=np.float32)

In [16]:
y_train_super = y_train_super.astype(np.float32)
y_val_super = y_val_super.astype(np.float32)
y_test_super = y_test_super.astype(np.float32)

# Defining Entropy and Metrics

In [17]:
import tensorflow.keras.backend as K

def weighted_binary_crossentropy(weights):
    def loss(y_true, y_pred):
        weights_cast = K.cast(weights, y_pred.dtype)
        y_true = K.cast(y_true, y_pred.dtype)
        
        bce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * weights_cast + (1 - y_true)
        weighted_bce = weight_vector * bce
        return K.mean(weighted_bce)
    return loss

def macro_f1(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    y_pred = K.round(y_pred)
    
    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# Defining Models

In [18]:
def create_cnn_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv1D(64, kernel_size=7, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(128, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    return model


In [19]:
# def create_resnet_model(input_shape, num_classes):
#     inputs = layers.Input(shape=input_shape)
#     x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    
#     previous_filters = x.shape[-1]
#     for filters in [64, 128, 256]:
#         x_shortcut = x
#         strides = 1
#         if previous_filters != filters:
#             strides = 2

#         x = layers.Conv1D(filters, kernel_size=3, strides=strides, padding='same')(x)
#         x = layers.BatchNormalization()(x)
#         x = layers.Activation('relu')(x)
#         x = layers.Conv1D(filters, kernel_size=3, padding='same')(x)
#         x = layers.BatchNormalization()(x)
        
#         if previous_filters != filters or strides != 1:
#             x_shortcut = layers.Conv1D(filters, kernel_size=1, strides=strides, padding='same')(x_shortcut)
#             x_shortcut = layers.BatchNormalization()(x_shortcut)
        
#         x = layers.Add()([x, x_shortcut])
#         x = layers.Activation('relu')(x)
#         previous_filters = filters
#     x = layers.GlobalAveragePooling1D()(x)
#     outputs = layers.Dense(num_classes, activation='sigmoid')(x)
#     model = models.Model(inputs, outputs)
#     return model

In [20]:
def residual_block_1d(x, filters, kernel_size=3, strides=1, downsample=False):
    shortcut = x
    
    x = layers.Conv1D(filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv1D(filters, kernel_size=kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def create_resnet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    layers_filters = [64, 128, 256, 512]
    layers_blocks = [3, 4, 6, 3]

    for filters, num_blocks in zip(layers_filters, layers_blocks):
        for i in range(num_blocks):
            if i == 0 and filters != x.shape[-1]:
                x = residual_block_1d(x, filters, strides=2, downsample=True)
            else:
                x = residual_block_1d(x, filters)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model

In [21]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def create_vit_model(input_shape, num_classes):
    patch_size = 10 
    num_patches = input_shape[0] // patch_size
    projection_dim = 64
    num_heads = 4
    transformer_layers = 8
    mlp_head_units = [256, 128]
    dropout_rate = 0.1

    inputs = layers.Input(shape=input_shape)
    x = layers.Reshape((num_patches, patch_size * input_shape[1]))(inputs)
    x = layers.Dense(units=projection_dim)(x)
    positions = tf.range(start=0, limit=num_patches, delta=1)
    position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)
    x = x + position_embedding(positions)
    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=dropout_rate
        )(x1, x1)
        x2 = layers.Add()([attention_output, x])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=[projection_dim * 2, projection_dim], dropout_rate=dropout_rate)
        x = layers.Add()([x3, x2])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Defining the training loop

In [22]:
def train_model(model, X_train, y_train, X_val, y_val, class_weight, batch_size=64, epochs=25):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', macro_f1]
    )
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        class_weight=class_weight
    )
    return history

# Training and Evaluating Models without CL

In [23]:
input_shape = X_train.shape[1:]
num_classes_super = y_train_super.shape[1]

cnn_super_model = create_cnn_model(input_shape, num_classes_super)
train_model(cnn_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

2024-11-19 16:09:31.270097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.549533: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.552110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.555656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other 

Epoch 1/25


2024-11-19 16:09:33.322857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-11-19 16:09:34.093169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-11-19 16:09:34.191615: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x73d9f8042e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-19 16:09:34.191677: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2024-11-19 16:09:34.242857: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-19 16:09:34.547318: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of 

267/267 [==============================] - 5s 7ms/step - loss: 0.2797 - accuracy: 0.6384 - macro_f1: 0.6584 - val_loss: 0.3618 - val_accuracy: 0.6342 - val_macro_f1: 0.6274 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2422 - accuracy: 0.6853 - macro_f1: 0.7098 - val_loss: 0.3546 - val_accuracy: 0.6696 - val_macro_f1: 0.6617 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2281 - accuracy: 0.7005 - macro_f1: 0.7294 - val_loss: 0.3061 - val_accuracy: 0.6873 - val_macro_f1: 0.7062 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 7ms/step - loss: 0.2208 - accuracy: 0.7081 - macro_f1: 0.7385 - val_loss: 0.3125 - val_accuracy: 0.6752 - val_macro_f1: 0.6934 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2149 - accuracy: 0.7142 - macro_f1: 0.7469 - val_loss: 0.3087 - val_accuracy: 0.6887 - val_macro_f1: 0.7001 - lr: 0.0010
Epoch 6/25
267/267 [======

In [24]:
resnet_super_model = create_resnet_model(input_shape, num_classes_super)
train_model(resnet_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

Epoch 1/25
267/267 [==============================] - 14s 22ms/step - loss: 0.3015 - accuracy: 0.6271 - macro_f1: 0.6294 - val_loss: 0.6704 - val_accuracy: 0.3882 - val_macro_f1: 0.4309 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2545 - accuracy: 0.6802 - macro_f1: 0.6948 - val_loss: 0.3448 - val_accuracy: 0.6561 - val_macro_f1: 0.6679 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2375 - accuracy: 0.6924 - macro_f1: 0.7167 - val_loss: 0.4898 - val_accuracy: 0.5415 - val_macro_f1: 0.6120 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2312 - accuracy: 0.6977 - macro_f1: 0.7230 - val_loss: 0.3757 - val_accuracy: 0.6789 - val_macro_f1: 0.6919 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2215 - accuracy: 0.7141 - macro_f1: 0.7379 - val_loss: 0.3218 - val_accuracy: 0.6664 - val_macro_f1: 0.6708 - lr: 0.0010
Epoch 6/2

In [25]:
vit_super_model = create_vit_model(input_shape, num_classes_super)
train_model(vit_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

Epoch 1/25
267/267 [==============================] - 13s 20ms/step - loss: 0.3824 - accuracy: 0.4987 - macro_f1: 0.4738 - val_loss: 0.3772 - val_accuracy: 0.5718 - val_macro_f1: 0.5947 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2926 - accuracy: 0.6282 - macro_f1: 0.6302 - val_loss: 0.3544 - val_accuracy: 0.6398 - val_macro_f1: 0.5986 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2587 - accuracy: 0.6657 - macro_f1: 0.6803 - val_loss: 0.3343 - val_accuracy: 0.6654 - val_macro_f1: 0.6820 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2377 - accuracy: 0.6905 - macro_f1: 0.7151 - val_loss: 0.3849 - val_accuracy: 0.6454 - val_macro_f1: 0.6654 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 20ms/step - loss: 0.2215 - accuracy: 0.7009 - macro_f1: 0.7318 - val_loss: 0.3230 - val_accuracy: 0.6827 - val_macro_f1: 0.6708 - lr: 0.0010
Epoch 6/2

In [26]:
num_classes_sub = y_train_sub.shape[1]
cnn_sub_model = create_cnn_model(input_shape, num_classes_sub)
train_model(cnn_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 3s 7ms/step - loss: 0.1084 - accuracy: 0.3936 - macro_f1: 0.1386 - val_loss: 0.1403 - val_accuracy: 0.5303 - val_macro_f1: 0.1263 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0899 - accuracy: 0.4710 - macro_f1: 0.1888 - val_loss: 0.1414 - val_accuracy: 0.5065 - val_macro_f1: 0.1767 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0840 - accuracy: 0.4974 - macro_f1: 0.2213 - val_loss: 0.1536 - val_accuracy: 0.4623 - val_macro_f1: 0.1564 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0775 - accuracy: 0.5103 - macro_f1: 0.2367 - val_loss: 0.1536 - val_accuracy: 0.3975 - val_macro_f1: 0.2138 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 7ms/step - loss: 0.0703 - accuracy: 0.5210 - macro_f1: 0.2643 - val_loss: 0.1270 - val_accuracy: 0.5163 - val_macro_f1: 0.2207 - lr: 0.0010
Epoch 6/25
267/

In [27]:
resnet_sub_model = create_resnet_model(input_shape, num_classes_sub)
train_model(resnet_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 14s 21ms/step - loss: 0.1181 - accuracy: 0.3439 - macro_f1: 0.0673 - val_loss: 0.1689 - val_accuracy: 0.4264 - val_macro_f1: 0.1547 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 20ms/step - loss: 0.1026 - accuracy: 0.4255 - macro_f1: 0.1100 - val_loss: 0.3509 - val_accuracy: 0.0974 - val_macro_f1: 0.0624 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0921 - accuracy: 0.4473 - macro_f1: 0.1355 - val_loss: 0.1577 - val_accuracy: 0.4567 - val_macro_f1: 0.1011 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0856 - accuracy: 0.4694 - macro_f1: 0.1732 - val_loss: 0.1486 - val_accuracy: 0.5140 - val_macro_f1: 0.1834 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0831 - accuracy: 0.4777 - macro_f1: 0.1829 - val_loss: 0.1678 - val_accuracy: 0.3551 - val_macro_f1: 0.1285 - lr: 0.0010
Epoch 6/2

In [28]:
vit_sub_model = create_vit_model(input_shape, num_classes_sub)
train_model(vit_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 13s 20ms/step - loss: 0.1335 - accuracy: 0.2333 - macro_f1: 0.0649 - val_loss: 0.1869 - val_accuracy: 0.2773 - val_macro_f1: 0.0707 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0894 - accuracy: 0.4011 - macro_f1: 0.1432 - val_loss: 0.1557 - val_accuracy: 0.4357 - val_macro_f1: 0.1488 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0701 - accuracy: 0.4798 - macro_f1: 0.2268 - val_loss: 0.1522 - val_accuracy: 0.4543 - val_macro_f1: 0.1872 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0587 - accuracy: 0.5259 - macro_f1: 0.2836 - val_loss: 0.1545 - val_accuracy: 0.4278 - val_macro_f1: 0.1870 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0489 - accuracy: 0.5568 - macro_f1: 0.3374 - val_loss: 0.1516 - val_accuracy: 0.5037 - val_macro_f1: 0.2110 - lr: 0.0010
Epoch 6/2

In [29]:
def evaluate_model(model, X_test, y_test, classes):
    y_pred = model.predict(X_test)
    y_pred_threshold = (y_pred >= 0.5).astype(int)
    report = classification_report(y_test, y_pred_threshold, target_names=classes, zero_division=0, output_dict=True)
    print(classification_report(y_test, y_pred_threshold, target_names=classes, zero_division=0))
    return report


In [30]:
print("CNN Superdiagnostic Classification Report:")
cnn_super_report = evaluate_model(cnn_super_model, X_test, y_test_super, classes_super)

print("ResNet Superdiagnostic Classification Report:")
resnet_super_report = evaluate_model(resnet_super_model, X_test, y_test_super, classes_super)

print("ViT Superdiagnostic Classification Report:")
vit_super_report = evaluate_model(vit_super_model, X_test, y_test_super, classes_super)


CNN Superdiagnostic Classification Report:
68/68 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          CD       0.84      0.68      0.75       496
         HYP       0.56      0.65      0.60       262
          MI       0.84      0.61      0.71       550
        NORM       0.83      0.88      0.86       963
        STTC       0.80      0.71      0.75       521

   micro avg       0.80      0.74      0.77      2792
   macro avg       0.77      0.71      0.73      2792
weighted avg       0.80      0.74      0.76      2792
 samples avg       0.77      0.76      0.75      2792

ResNet Superdiagnostic Classification Report:
68/68 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

          CD       0.81      0.69      0.75       496
         HYP       0.72      0.50      0.59       262
          MI       0.72      0.77      0.74       550
        NORM       0.87      0.82      0.84  

In [31]:
print("CNN Subdiagnostic Classification Report:")
cnn_sub_report = evaluate_model(cnn_sub_model, X_test, y_test_sub, classes_sub)

print("ResNet Subdiagnostic Classification Report:")
resnet_sub_report = evaluate_model(resnet_sub_model, X_test, y_test_sub, classes_sub)

print("ViT Subdiagnostic Classification Report:")
vit_sub_report = evaluate_model(vit_sub_model, X_test, y_test_sub, classes_sub)


CNN Subdiagnostic Classification Report:
68/68 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         AMI       0.88      0.53      0.66       306
       CLBBB       0.91      0.91      0.91        54
       CRBBB       0.79      0.91      0.84        54
       ILBBB       0.12      0.12      0.12         8
         IMI       0.79      0.48      0.60       327
       IRBBB       0.56      0.68      0.62       112
        ISCA       0.54      0.16      0.25        93
        ISCI       0.44      0.30      0.36        40
        ISC_       0.72      0.47      0.57       128
        IVCD       0.19      0.06      0.09        79
   LAFB/LPFB       0.80      0.64      0.71       179
     LAO/LAE       0.00      0.00      0.00        42
         LMI       0.30      0.15      0.20        20
         LVH       0.75      0.53      0.62       214
        NORM       0.88      0.76      0.81       963
        NST_       0.32      0.16      0.